# **ECF Declaration Forecast**

**Purpose:**

To produce a table of actual and forecast declaration volumes per statement in the open cohorts. Currently open cohorts in scope are 2023 & 2024.

**Summary:**

By using 3 key metrics from the available historic data across cohorts 2021, 2022 & 2023 this notebook looks to produce a forecast of declarations per statement broken down by participant type (ECT or Mentor) and Declaration Type (Started through to Completed)

The 3 key metrics calculated and used are:
- Participant Volumes
- Declaration Volumes
- Declaration Landing Points

**Developer Notes:**

 The 2021 Statement Cohort is not being used for when declarations were paid due to the inconsistent nature of the statement periods. It is being used for overall declaration volume however.

## **File Preparation**

### Import required libraries

In [1]:
# Import required libraries
import pandas as pd, numpy as np, datetime
from google.cloud import bigquery

client = bigquery.Client()

### Set Pandas Options

In [2]:
# Set Pandas Options
## Limitless Rows
pd.set_option('display.max_rows', None)
## Limitless Columns
pd.set_option('display.max_columns', None)

### Define Statement Cohort to Statement Interval Mappings

**Output:** `statement_mappings`

**Description:**

The output from this section creates a dataframe containing the `statement_cohort`, `statement_date`, `statement_cut_off_date` and `statement_number` for cohorts 2021 to 2024. An additional column named `statement_has_passed` is added that determines if  a statement in each cohort has passed whereby the cut-off date is before today's date.

The main purpose of this dataframe is to lay the foundation onto which actual and projected values for the key metrics can aggregated consistently to.

**Notes:**
The statement mapping for 2021 is not realistic and only used so that it aligns with the other cohorts. This allows us to aggregate data at the same points in time but for 2021 but only Participant Volumes are measured by the dates referenced within.

In [3]:
statement_mapping_2021 = {
    'statement_cohort': 2021,
    'statement_date': [
        pd.to_datetime('2021-11-01'),
        pd.to_datetime('2022-01-01'),
        pd.to_datetime('2022-04-01'),
        pd.to_datetime('2022-08-01'),
        pd.to_datetime('2023-01-01'),
        pd.to_datetime('2023-04-01'),
        pd.to_datetime('2023-08-01'),
        pd.to_datetime('2024-01-01'),
        pd.to_datetime('2024-04-01'),
        pd.to_datetime('2024-08-01'),
        pd.to_datetime('2025-01-01')
        ],
    'statement_cut_off_date': [
        pd.to_datetime('2021-10-31'),
        pd.to_datetime('2021-12-31'),
        pd.to_datetime('2022-03-31'),
        pd.to_datetime('2022-07-31'),
        pd.to_datetime('2022-12-31'),
        pd.to_datetime('2023-03-31'),
        pd.to_datetime('2023-07-31'),
        pd.to_datetime('2023-12-31'),
        pd.to_datetime('2024-03-31'),
        pd.to_datetime('2024-07-31'),
        pd.to_datetime('2024-12-31')
        ],
    'statement_number': [1,2,3,4,5,6,7,8,9,10,11]
}

statement_mapping_2022 = {
    'statement_cohort': 2022,
    'statement_date': [
        pd.to_datetime('2022-11-01'),
        pd.to_datetime('2023-01-01'),
        pd.to_datetime('2023-04-01'),
        pd.to_datetime('2023-08-01'),
        pd.to_datetime('2024-01-01'),
        pd.to_datetime('2024-04-01'),
        pd.to_datetime('2024-08-01'),
        pd.to_datetime('2025-01-01'),
        pd.to_datetime('2025-04-01'),
        pd.to_datetime('2025-08-01'),
        pd.to_datetime('2026-01-01')
        ],
    'statement_cut_off_date': [
        pd.to_datetime('2022-10-31'),
        pd.to_datetime('2022-12-31'),
        pd.to_datetime('2023-03-31'),
        pd.to_datetime('2023-07-31'),
        pd.to_datetime('2023-12-31'),
        pd.to_datetime('2024-03-31'),
        pd.to_datetime('2024-07-31'),
        pd.to_datetime('2024-12-31'),
        pd.to_datetime('2025-03-31'),
        pd.to_datetime('2025-07-31'),
        pd.to_datetime('2025-12-31')
        ],
    'statement_number': [1,2,3,4,5,6,7,8,9,10,11]
}

statement_mapping_2023 = {
    'statement_cohort': 2023,
    'statement_date': [
        pd.to_datetime('2023-11-01'),
        pd.to_datetime('2024-01-01'),
        pd.to_datetime('2024-04-01'),
        pd.to_datetime('2024-08-01'),
        pd.to_datetime('2025-01-01'),
        pd.to_datetime('2025-04-01'),
        pd.to_datetime('2025-08-01'),
        pd.to_datetime('2026-01-01'),
        pd.to_datetime('2026-04-01'),
        pd.to_datetime('2026-08-01'),
        pd.to_datetime('2027-01-01')
        ],
    'statement_cut_off_date': [
        pd.to_datetime('2023-10-31'),
        pd.to_datetime('2023-12-31'),
        pd.to_datetime('2024-03-31'),
        pd.to_datetime('2024-07-31'),
        pd.to_datetime('2024-12-31'),
        pd.to_datetime('2025-03-31'),
        pd.to_datetime('2025-07-31'),
        pd.to_datetime('2025-12-31'),
        pd.to_datetime('2026-03-31'),
        pd.to_datetime('2026-07-31'),
        pd.to_datetime('2026-12-31')
        ],
    'statement_number': [1,2,3,4,5,6,7,8,9,10,11]
}

statement_mapping_2024 = {
    'statement_cohort': 2024,
    'statement_date': [
        pd.to_datetime('2024-11-01'),
        pd.to_datetime('2025-01-01'),
        pd.to_datetime('2025-04-01'),
        pd.to_datetime('2025-08-01'),
        pd.to_datetime('2026-01-01'),
        pd.to_datetime('2026-04-01'),
        pd.to_datetime('2026-08-01'),
        pd.to_datetime('2027-01-01'),
        pd.to_datetime('2027-04-01'),
        pd.to_datetime('2027-08-01'),
        pd.to_datetime('2028-01-01')
        ],
    'statement_cut_off_date': [
        pd.to_datetime('2024-10-31'),
        pd.to_datetime('2024-12-31'),
        pd.to_datetime('2025-03-31'),
        pd.to_datetime('2025-07-31'),
        pd.to_datetime('2025-12-31'),
        pd.to_datetime('2026-03-31'),
        pd.to_datetime('2026-07-31'),
        pd.to_datetime('2026-12-31'),
        pd.to_datetime('2027-03-31'),
        pd.to_datetime('2027-07-31'),
        pd.to_datetime('2027-12-31')
        ],
    'statement_number': [1,2,3,4,5,6,7,8,9,10,11]
}

statement_mappings = pd.concat([
    pd.DataFrame(statement_mapping_2021),
    pd.DataFrame(statement_mapping_2022),
    pd.DataFrame(statement_mapping_2023),
    pd.DataFrame(statement_mapping_2024)
]).reset_index()

statement_mappings['statement_has_passed'] = np.where(statement_mappings['statement_cut_off_date'].dt.date <= datetime.date.today(), True, False)



### Load Required Data

#### Load Latest Paid Declarations per Statement

**Output:** `paid_declarations`

**Description:**

The output from this section creates a dataframe containing all funded declarations (e/p/p) to date in their latest state. Any voided declarations will not appear in this output.

This list of declarations is to be used as the latest position and does not allow for assessing initial payment point, clawback rate or timing, nor resubmission rates.

**Notes:**

Extended declarations are re-sequenced in this section due to there being a higher proportion of extended-3's in the service data compared to extended-1's or 2's. This is achieved by assessing per participant the order of extended declarations based on their `created_at` date and numbering them 1 to 3 depending on how many have been created.

In [4]:
# Load latest paid declarations per statment
query = """
  WITH latest_declarations AS (
  SELECT
     declarations.id AS declaration_id
    -- ,declarations.cpd_lead_provider_id AS declaration_provider_id
    ,declarations.created_at AS declaration_created_at
    ,declarations.declaration_date
    ,declarations.declaration_type
    ,CASE
      WHEN declaration_type = 'started' THEN 1
      WHEN declaration_type = 'retained-1' THEN 2
      WHEN declaration_type = 'retained-2' THEN 3
      WHEN declaration_type = 'retained-3' THEN 4
      WHEN declaration_type = 'retained-4' THEN 5
      WHEN declaration_type = 'extended-1' THEN 6
      WHEN declaration_type = 'extended-2' THEN 7
      WHEN declaration_type = 'extended-3' THEN 8
      WHEN declaration_type = 'completed' THEN 9
      ELSE 0
    END AS declaration_sequence
    ,declarations.state AS declaration_state
    ,declarations.participant_profile_id
    ,declarations.user_id
  FROM
    `dataform.participant_declarations_latest_cpd` AS declarations
  WHERE
    declarations.type NOT LIKE '%NPQ%'
    AND
    declarations.state IN ('eligible', 'payable', 'paid')
),

latest_declarations_extended AS (
  SELECT
     dec.*
    ,CASE
      WHEN inductions.participant_type LIKE '%ECT' THEN 'ECT'
      WHEN inductions.participant_type LIKE '%Mentor' THEN 'Mentor'
      ELSE NULL
     END AS participant_type
    ,inductions.cohort_rolled_over_21_24 AS participant_rolled_over_to_24
  FROM
    latest_declarations AS dec
  INNER JOIN
    `dataform.ecf_inductions_dedupe` AS inductions
  ON
    inductions.participant_profile_id = dec.participant_profile_id
),

output AS (
  SELECT
     stm.id AS statement_id
    ,stm.name AS statement_name
    ,PARSE_DATE('%B %Y', stm.name) AS statement_date
    ,stm.type AS statement_type
    -- ,stm.cpd_lead_provider_id AS statement_provider_id
    ,cohorts.start_year AS statement_cohort
    , dec.*
  FROM
    latest_declarations_extended AS dec
  INNER JOIN
    `dataform.statement_line_items_latest_cpd` AS sli
  ON
    sli.participant_declaration_id = dec.declaration_id
    AND
    sli.state IN ('eligible', 'payable', 'paid')
  LEFT JOIN
    `dataform.statements_latest_cpd` AS stm
  ON
    stm.id = sli.statement_id
  LEFT JOIN
    `dataform.cohorts_latest_cpd` AS cohorts
  ON
    cohorts.id = stm.cohort_id
  QUALIFY(
    ROW_NUMBER() OVER(PARTITION BY declaration_id ORDER BY sli.created_at DESC) = 1
  )
)

SELECT
   *
FROM
  output
"""

paid_declarations = client.query(query).to_dataframe()
paid_declarations['statement_date'] = paid_declarations['statement_date'].astype('datetime64[ns]')

## Re-Sequence the extended declarations based on when they were created.
paid_declarations.loc[paid_declarations['declaration_type'].str.startswith('extended'), 'extended_sequence'] = (
    paid_declarations[paid_declarations['declaration_type'].str.startswith('extended')]
      .sort_values(by='declaration_created_at')
      .groupby('participant_profile_id')
      .cumcount() + 1
)

paid_declarations.fillna({'extended_sequence':0}, inplace=True)
paid_declarations['extended_sequence'] = paid_declarations['extended_sequence'].astype(int)

paid_declarations.loc[paid_declarations['declaration_type'].str.startswith('extended'), 'declaration_type'] = (
    'extended-'
    +
    paid_declarations[paid_declarations['declaration_type'].str.startswith('extended')]['extended_sequence'].astype(str)
)

dec_order = {
    'started':1,
    'retained-1':2,
    'retained-2':3,
    'retained-3':4,
    'retained-4':5,
    'extended-1':6,
    'extended-2':7,
    'extended-3':8,
    'completed':9
}

paid_declarations['declaration_sequence'] = paid_declarations['declaration_type'].map(dec_order)



#### Load Participant Volumes

**Output:** `participant_volumes`

**Description:**

The output from this section is a dataframe containing the number of participants & eligible for funding participants with an active induction record per statement cut-off date. These volumes are also aggregated at `cohort` and `participant_type`.

This dataframe is to be used for measuring the number of participants at each point in time as well as the percentage of participants expected to have been registered per statement based on the maximum value present per grouping of cohort and participant type.

In [5]:
# Define Get Participant Volumes Function
def get_participant_volumes(snapshot_date):
  query = f"""
    SELECT
      '{snapshot_date}' AS snapshot_date
      ,cohort
      ,CASE
        WHEN participant_type LIKE '%Mentor' THEN 'Mentor'
        WHEN participant_type LIKE '%ECT' THEN 'ECT'
        ELSE NULL
       END AS participant_type
      ,COUNT(DISTINCT participant_profile_id) AS participants
      ,COUNT(DISTINCT CASE WHEN eligible_for_funding = TRUE THEN participant_profile_id ELSE NULL END) AS eligible_participants
    FROM
      `dataform.ecf_inductions`
    WHERE
      participant_profile_id IS NOT NULL
      AND
      cohort IS NOT NULL
      AND
      '{snapshot_date}' BETWEEN DATE(start_date) AND IFNULL(DATE(end_date), DATE(2050,12,31))
    GROUP BY ALL
    ORDER BY cohort, participant_type
  """

  df = client.query(query).to_dataframe()

  return df

# Gather list of statement cut-off dates from the statement mappings DF.
dates = statement_mappings['statement_cut_off_date'].dt.date.unique()

# Loop through these dates and call the get_participant_volumes function
dfs = []
for date in dates:
    # print(date)
  df = get_participant_volumes(date)
  dfs.append(df)

# Concatenate all gathered DataFrames together
participant_volumes = pd.concat(dfs)

# Change snapshot_date column to a datetime data type.
participant_volumes['snapshot_date'] = participant_volumes['snapshot_date'].astype('datetime64[ns]')

## **Definitions**

### Participant Volumes

#### Maximum Participant Volumes per Cohort

**Output:** `max_participant_volumes`

**Description:**

This section uses the `participant_volumes` dataframe loaded earlier to determine per cohort the highest volume of eligible participants seen across the snapshots per participant type.

In [16]:
max_participant_volumes = (
    participant_volumes
    .groupby(['cohort', 'participant_type'])
    .agg(max_participant_count = ('eligible_participants', 'max'))
    .reset_index()
)

#### Statement Number Participant Volume Percentage

**Output:** `statement_participant_volume_percentage`

**Description:**

This section generates a dataframe containing the `statement_number`, `participant_type`, `avg_participant_percentage` and `adj_participant_percentage`.

`avg_participant_percentage` is calculated by:
- Determining the eligible participant percentage per snapshot per cohort & participant type as a percentage of the maximum value seen per cohort and participant type.
- These percentages are then averaged (mean) per statement number. Example Scenario: \
ECTs statement 1:
2021 = 92%, 2022 = 93%, 2023 = 94%; avg_participant_percentage = 93%

None of the averaged percentages will be equal to 100%, therefore to avoid missing participants from the eventual forecast there needs to be an adjustment made to these average figures where the new maximum is the statement number with the highest average participant volume per cohort-participant type combination. This is achieved by:
- Determining the new maximum percentage and adjusting this to 100%
- Transform the avg_participant_percentage into a new percentage reflective of the change to 100% for the maximum.

In [17]:
# Combine statement mappings with participant volumes on the statement cut-off date.
statement_participant_mapping = (
    statement_mappings
    .merge(participant_volumes, how='left', left_on=['statement_cohort', 'statement_cut_off_date'], right_on=['cohort', 'snapshot_date'])
    .drop(columns=['cohort', 'snapshot_date'])
)

# Merge on max participant volumes
statement_participant_volume_percentage = (
    statement_participant_mapping
    .merge(max_participant_volumes, how='left', left_on=['statement_cohort', 'participant_type'], right_on=['cohort', 'participant_type'])
)
# Add a column for the percentage of the maximum that the volume equals
statement_participant_volume_percentage['participant_percentage'] = round(
    (statement_participant_volume_percentage['eligible_participants'] / statement_participant_volume_percentage['max_participant_count']) * 100, 2
)
# Remove redundant columns
statement_participant_volume_percentage.drop(columns=['max_participant_count', 'index'], inplace=True)

# Take an average per statement number and participant type for only the statements that have passed.
statement_participant_volume_percentage = (
    statement_participant_volume_percentage
    .loc[statement_participant_volume_percentage['statement_has_passed']==True]
    .groupby(['statement_number', 'participant_type'])
    .agg(avg_participant_percentage = ('participant_percentage','mean'))
    .reset_index()
)

# Adjust the average percentages to be a proportion of 100% where the highest percentage is converted to 100%.
statement_participant_volume_percentage['adj_participant_percentage'] = round(
    (statement_participant_volume_percentage['avg_participant_percentage'] /
    statement_participant_volume_percentage.groupby(['participant_type'])['avg_participant_percentage'].transform('max')) * 100, 2
)

In [18]:
statement_participant_volume_percentage

,statement_number,participant_type,avg_participant_percentage,adj_participant_percentage
0,1,ECT,89.895,90.25
1,1,Mentor,94.2875,94.33
2,2,ECT,91.415,91.77
3,2,Mentor,96.19,96.23
4,3,ECT,99.583333,99.97
5,3,Mentor,99.96,100.0
6,4,ECT,99.106667,99.49
7,4,Mentor,99.913333,99.95
8,5,ECT,99.036667,99.42
9,5,Mentor,98.633333,98.67


### Declaration Volumes

#### Declaration Volume per Statement Cohort & Participant type

**Output:** `statement_declaration_volumes`

**Description:**

The output from this section uses the `paid_declarations` DF loaded earlier containing all latest paid declarations and aggregates the volume of declarations per `cohort`, `participant_type`, `declaration_sequence` and `declaration_type`.

This output will be used to calculate the average expected declaration volume per cohort, participant type and declaration type combination.

In [19]:
statement_declaration_volumes = (
    paid_declarations
    .groupby([
        'statement_cohort',
        'participant_type',
        'declaration_sequence',
        'declaration_type'
      ])
    .agg(total_declaration_count = ('declaration_id', 'nunique'))
    .reset_index()
)

In [20]:
statement_declaration_volumes

,statement_cohort,participant_type,declaration_sequence,declaration_type,total_declaration_count
0,2021,ECT,1,started,26955
1,2021,ECT,2,retained-1,26456
2,2021,ECT,3,retained-2,25463
3,2021,ECT,4,retained-3,24564
4,2021,ECT,5,retained-4,22274
5,2021,ECT,6,extended-1,602
6,2021,ECT,7,extended-2,294
7,2021,ECT,8,extended-3,93
8,2021,ECT,9,completed,22238
9,2021,Mentor,1,started,25844


#### Statement Number Declaration Volume Percentage

**Output:** `statement_declaration_volume_percentage`

**Description:**

The output from this section contains the `avg_declaration_percentage` and `adj_declaration_percentage` per `statement_number` for each `cohort`, `participant_type` & `declaration_type` combination.

The `avg_declaration_percentage` is calculated by:
- Group all paid declarations by statement cohort, statement date, participant type and declaration type.
- Calculate the percentage these equate to for all declarations paid per cohort-participant type-declaration type combination to have a declaration percentage per statement per cohort.
- Take the average (mean) of these percentages at the statement number level

The sum of these averages will not equal 100% therefore an adjusted percentage is required to achieve this. The `adj_declaration_percentage` is achieved by calculating the proportion of the total average percentage that the value equals. Example: \
ECT Started Declarations in Statement 1 average % = 68.51%. \
Sum of ECT Started Declaration average percentages = 100.75%
Adjusted Percentage = 68% (68.51 / 100.75)

**Notes:**

Only statement cohorts 2022 & 2023 are currently being used for when declarations were paid. Also, only statements that have passed are being used. This means that as more statements pass this variable will update to include those in its computations.

A restiction is also added that it only uses declaration type total declaration values where there are 5,000 or more to avoid skewing of the data in more recent statements. This is not applied to extended declarations as these volumes are usually very low.

In [21]:
# Joining Statement Mappings with grouped Paid Declarations
## Grouping is applied to Statement Cohort, Statement Date, Participant Type and Declaration Type
statement_mappings_extended = (
    statement_mappings
    .merge(
        (
            paid_declarations
            .groupby(['statement_cohort', 'statement_date', 'participant_type', 'declaration_sequence', 'declaration_type'])
            .agg(declaration_count = ('declaration_id', 'nunique'))
            .reset_index()
        ),
        how='left',
        on=['statement_cohort', 'statement_date'])
)

# Merge declaration volumes onto the extended statement mappings.
## Statement Cohorts are set to 2022 & 2023 only as well as only the statements that have passed.
statement_declaration_volume_percentage = (
    statement_mappings_extended
    .loc[
        (statement_mappings_extended['statement_cohort'].isin([2022,2023])) &
        (statement_mappings_extended['statement_has_passed']==True)
         ]
    .merge(statement_declaration_volumes,
           how='left',
           on=['statement_cohort', 'participant_type', 'declaration_sequence', 'declaration_type']
    )
)

## Filter out any values where the declaration type is not extended_# and the total declaration count on which the percentage is being calculated is less that 5_000.
statement_declaration_volume_percentage = (
    statement_declaration_volume_percentage.loc[
        (statement_declaration_volume_percentage['total_declaration_count']>= 5_000) |
        (statement_declaration_volume_percentage['declaration_type'].str.startswith('extended'))
    ]
)

# Add column for declaration percentage which is the row level value divided by the total number.
statement_declaration_volume_percentage['declaration_percentage'] = round(
    (statement_declaration_volume_percentage['declaration_count'] /
    statement_declaration_volume_percentage['total_declaration_count']) * 100, 3
)

# Calculate the average declaration percentage per statement number, participant type & declaration type
statement_declaration_volume_percentage = (
    statement_declaration_volume_percentage
    .groupby(['statement_number', 'participant_type', 'declaration_sequence', 'declaration_type'])
    .agg(avg_declaration_percentage = ('declaration_percentage', 'mean'))
    .reset_index()
)

# Add column that adjusts the average percentage to being a proportion of the average percentage total.
## This is required as the sum of average percentages will not equal 100%.
statement_declaration_volume_percentage['adj_declaration_percentage'] = round(
    (statement_declaration_volume_percentage['avg_declaration_percentage'] /
    statement_declaration_volume_percentage.groupby(['participant_type', 'declaration_type'])['avg_declaration_percentage'].transform('sum')) * 100, 3
)

In [22]:
statement_declaration_volume_percentage[(statement_declaration_volume_percentage['participant_type']=='ECT') & (statement_declaration_volume_percentage['declaration_type']=='started')]

,statement_number,participant_type,declaration_sequence,declaration_type,avg_declaration_percentage,adj_declaration_percentage
0,1,ECT,1.0,started,68.4300,68.000
4,2,ECT,1.0,started,15.3700,15.273
6,3,ECT,1.0,started,10.2065,10.142
12,4,ECT,1.0,started,4.1910,4.165
20,5,ECT,1.0,started,0.9265,0.921
31,6,ECT,1.0,started,1.1640,1.157
44,7,ECT,1.0,started,0.2290,0.228
58,8,ECT,1.0,started,0.1150,0.114


### Participant Declaration Proportions

#### Cohort Declaration Proportions

**Output:** `cohort_declaration_proportions`

**Description:**

The output from this section contains `cohort`, `participant_type`, `eligible_participants` and a column for each declaration type containing the number of declarations paid/payable or eligible. Added to this is the percentage of eligible participants that recieved a declaration of each type.

In [23]:
cohort_declaration_proportions = (
    max_participant_volumes.copy().rename(columns={'max_participant_count':'eligible_participants'})
)

# Started Declaration Counts
started_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='started')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'started_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Retained-1 Declaration Counts
ret_1_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='retained-1')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'retained_1_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Retained-2 Declaration Counts
ret_2_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='retained-2')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'retained_2_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Retained-3 Declaration Counts
ret_3_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='retained-3')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'retained_3_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Retained-4 Declaration Counts
ret_4_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='retained-4')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'retained_4_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Extended-1 Declaration Counts
ext_1_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='extended-1')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'extended_1_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Extended-2 Declaration Counts
ext_2_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='extended-2')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'extended_2_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Extended-3 Declaration Counts
ext_3_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='extended-3')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'extended_3_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Completed Declaration Counts
completed_decs = (
    statement_declaration_volumes
    .loc[(statement_declaration_volumes['declaration_type']=='completed')]
    .groupby(['statement_cohort','participant_type'])
    .agg({'total_declaration_count':'sum'})
    .reset_index()
    .rename(columns={
        'total_declaration_count':'completed_declaration_count',
        'statement_cohort':'cohort'
        })
)

# Combine all declaration counts with the eligible participant counts
cohort_declaration_proportions = (
    cohort_declaration_proportions
    .merge(started_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ret_1_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ret_2_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ret_3_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ret_4_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ext_1_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ext_2_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(ext_3_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .merge(completed_decs,
           how='left',
           on=['cohort', 'participant_type'])
    .fillna(0)
)

# Add percentage proportions
cohort_declaration_proportions = (
    cohort_declaration_proportions
    .assign(**{
        col.replace('count', 'percentage'): round(
        (cohort_declaration_proportions[col] / cohort_declaration_proportions['eligible_participants'])*100, 2)
        for col in cohort_declaration_proportions.iloc[:, 3:]
    })
)

#### Average Declaration Proportions

**Output:** `avg_declaration_proportions`

**Description:**

The output of this section contains `participant_type`, `declaration_type` and the average percentage that recieve that declaration type.

Started, Retained-1, 2 & 3 use cohorts 2022 & 2023 while Retained-4, Extended-1,2,3 and Completed only use cohort 2022.

**Notes:**

As there is no firm point in time by which a cohort is ready to be used for particular declaration types, there is no means of automatically including additional cohorts. Once it is appropriate, the filters can be modified accordingly to use other cohorts.

In [24]:
avg_started_proportion = (
    cohort_declaration_proportions[['participant_type', 'started_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2024]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('started_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'started'})
    .reset_index()
)

avg_ret1_proportion = (
    cohort_declaration_proportions[['participant_type', 'retained_1_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2024]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('retained_1_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'retained-1'})
    .reset_index()
)

avg_ret2_proportion = (
    cohort_declaration_proportions[['participant_type', 'retained_2_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2024]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('retained_2_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'retained-2'})
    .reset_index()
)

avg_ret3_proportion = (
    cohort_declaration_proportions[['participant_type', 'retained_3_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2024]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('retained_3_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'retained-3'})
    .reset_index()
)

avg_ret4_proportion = (
    cohort_declaration_proportions[['participant_type', 'retained_4_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2023]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('retained_4_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'retained-4'})
    .reset_index()
)

avg_ext1_proportion = (
    cohort_declaration_proportions[['participant_type', 'extended_1_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<=2023]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('extended_1_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'extended-1'})
    .reset_index()
)

avg_ext2_proportion = (
    cohort_declaration_proportions[['participant_type', 'extended_2_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<=2023]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('extended_2_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'extended-2'})
    .reset_index()
)

avg_ext3_proportion = (
    cohort_declaration_proportions[['participant_type', 'extended_3_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<=2023]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('extended_3_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'extended-3'})
    .reset_index()
)

avg_completed_proportion = (
    cohort_declaration_proportions[['participant_type', 'completed_declaration_percentage']]
    .loc[cohort_declaration_proportions['cohort']<2023]
    .groupby('participant_type')
    .agg(avg_declaration_proportion = ('completed_declaration_percentage', 'mean'))
    .assign(**{'declaration_type': 'completed'})
    .reset_index()
)

avg_declaration_proportions = pd.concat([
    avg_started_proportion,
    avg_ret1_proportion,
    avg_ret2_proportion,
    avg_ret3_proportion,
    avg_ret4_proportion,
    avg_ext1_proportion,
    avg_ext2_proportion,
    avg_ext3_proportion,
    avg_completed_proportion
]).reset_index().drop(columns='index')

### Participant Attrition

Within this section we calculate the change in participant volumes per cohort and participant course to determine a rate of attrition for ECTs and Mentors that can be used as a method to predict a future cohort's participant volume.

**DataFrames Defined:** \
`ect_attrition` \
`mentor_attrition` \

**Variables Defined:** \
`ect_avg_attrition`: float \
`mentor_avg_attrition`: float

In [25]:
# Create DataFrame of eligible registered participant volumes per cohort and participant type.

# Create DataFrame of ECTs values only.
## 2024 Cohort is excluded until statement 3 (April 2025) has passed
if datetime.datetime.today() < pd.to_datetime('2025-04-01'):
  ect_attrition = (
      max_participant_volumes
      .loc[
          (max_participant_volumes['participant_type']=='ECT') &
          (max_participant_volumes['cohort'] < 2024)
          ]
      .copy()
      .sort_values(by='cohort')
  )
else:
  ect_attrition = (
      max_participant_volumes
      .loc[
          (max_participant_volumes['participant_type']=='ECT') &
          (max_participant_volumes['cohort'] <= 2024)
          ]
      .copy()
      .sort_values(by='cohort')
  )
## Calculate the change percentage between cohorts.
ect_attrition.loc[:, 'cohort_change'] = (
    (
        ect_attrition.max_participant_count
        /
        ect_attrition.max_participant_count.shift(1)
    )
    -1
)
## Fill the NA value for cohort 2021 due to no preceeding data.
ect_attrition = ect_attrition.fillna(0)

display(ect_attrition)

# Create DataFrame of Mentor values only.
## 2024 Cohort is excluded until statement 3 (April 2025) has passed
if datetime.datetime.today() < pd.to_datetime('2025-04-01'):
  mentor_attrition = (
      max_participant_volumes
      .loc[
          (max_participant_volumes['participant_type']=='Mentor') &
          (max_participant_volumes['cohort'] < 2024)
          ]
      .copy()
      .sort_values(by='cohort')
  )
else:
  mentor_attrition = (
      max_participant_volumes
      .loc[
          (max_participant_volumes['participant_type']=='Mentor') &
          (max_participant_volumes['cohort'] <= 2024)
          ]
      .copy()
      .sort_values(by='cohort')
  )
## Calculate the change percentage between cohorts.
mentor_attrition.loc[:, 'cohort_change'] = (
    (
        mentor_attrition.max_participant_count
        /
        mentor_attrition.max_participant_count.shift(1)
    )
    -1
)
## Fill the NA value for cohort 2021 due to no preceeding data.
mentor_attrition = mentor_attrition.fillna(0)

display(mentor_attrition)

ect_avg_attrition = round(ect_attrition[ect_attrition['cohort_change']!= 0]['cohort_change'].mean(), 4)

mentor_avg_attrition = round(mentor_attrition[mentor_attrition['cohort_change'] != 0]['cohort_change'].mean(), 4)

# Show output for reference: Average Cohort Attrition for ECTs and Mentors.
print(f'ECT Attrition Avg: {ect_avg_attrition}')
print(f'Mentor Attrition Avg: {mentor_avg_attrition}')

,cohort,participant_type,max_participant_count,cohort_change
0,2021,ECT,29274,0.0
2,2022,ECT,28994,-0.009565
4,2023,ECT,24653,-0.149721


,cohort,participant_type,max_participant_count,cohort_change
1,2021,Mentor,33328,0.0
3,2022,Mentor,25938,-0.221735
5,2023,Mentor,22183,-0.144768


ECT Attrition Avg: -0.0796
Mentor Attrition Avg: -0.1833


## **Processing**

This section contains all of the processing in order to generate a forecast for a given cohort.

### Cohort Forecast Functions

#### Filter required dataframes

**Output:** Function `get_filtered_data(cohort)`

**Description:**

This section defines the above function that requires a cohort to be provided. It then returns 3 dataframes:\
[0] `statement_mappings` filtered to the provided statement cohort.\
[1] `statement_participant_mapping` filtered to the provided cohort.\
[2] `paid_declarations` filtered to the provided statement cohort and grouped by statement date, participant type and declaration type.


In [ ]:
def get_filtered_data(cohort):
  statements = statement_mappings[statement_mappings['statement_cohort']==cohort].drop(columns='index')

  act_participants = statement_participant_mapping[statement_participant_mapping['statement_cohort']==cohort]

  act_declarations = (
      paid_declarations
      .loc[paid_declarations['statement_cohort']==cohort]
      .groupby(['statement_date', 'participant_type', 'declaration_sequence', 'declaration_type'])
      .agg(act_declaration_count = ('declaration_id', 'nunique'))
      .reset_index()
      .merge(statements[['statement_date', 'statement_number']], how='left', on='statement_date')
  )

  return statements, act_participants, act_declarations

#### Define Function to get actual participant counts per Statement Number

**Output:** Function `get_actual_participant_counts(stm_number)`

**Description:**

The output of the above function which requires a statement number to be passed in, is a dataframe containing the actual participant count as at the cut-off date for the statement number provided.

It uses two dataframes named `statements` and `act_participants` which must be instantiated before calling this function.

In [ ]:
def get_actual_participant_counts(stm_number):
  df = (
    statements
    .loc[statements['statement_number']==stm_number]
    .merge(act_participants[['statement_number', 'participant_type', 'eligible_participants']], how='left', on='statement_number')
    .rename(columns={'eligible_participants':'actual_participants'})
  )

  return df

#### Define Function to get actual declarations per statement number

**Output:** Function `get_actual_declarations(df, stm_number)`

**Description:**

The function defined within this section is used within another called `generate_statement_dataframe(stm_number)` and as such should not be called in isolation.

It requires a DataFrame (df) and a Statement Number to be passed into it and also relies on a dataframe named `act_declarations` which must be instantiated before calling.

It returns the supplied dataframe with additional columns for each declaration type and the number recieved on the provided statement number.

In [ ]:
def get_actual_declarations(df, stm_number):
  declarations = act_declarations[act_declarations['statement_number']==stm_number]

  started_decs = declarations[declarations['declaration_type']=='started']
  ret1_decs = declarations[declarations['declaration_type']=='retained-1']
  ret2_decs = declarations[declarations['declaration_type']=='retained-2']
  ret3_decs = declarations[declarations['declaration_type']=='retained-3']
  ret4_decs = declarations[declarations['declaration_type']=='retained-4']
  ext1_decs = declarations[declarations['declaration_type']=='extended-1']
  ext2_decs = declarations[declarations['declaration_type']=='extended-2']
  ext3_decs = declarations[declarations['declaration_type']=='extended-3']
  completed_decs = declarations[declarations['declaration_type']=='completed']

  df = (
      df
      .merge(
        started_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_started_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ret1_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ret1_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ret2_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ret2_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ret3_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ret3_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ret4_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ret4_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ext1_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ext1_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ext2_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ext2_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        ext3_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_ext3_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .merge(
        completed_decs[['statement_number',
                      'participant_type',
                      'act_declaration_count'
                    ]].rename(columns={'act_declaration_count': 'act_completed_declaration_count'}),
        how='left',
        on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )

  return df

#### Define Function for estimating participant counts per Statement Number

**Outputs:**
Functions\
`participant_volume_forecast(stm_number)`\
`estimate_participant_count(df, stm_number)`

**Description:**

Within this section 2 functions are defined. The first `participant_volume_forecast` requires a statement number to be passed in and for a dataframe named `exp_participants` to have been instantiated. This function is used within the second `estimate_participant_count` and should not be called independatly. It is used when there is no previous statment number values to account for.

The second function, `estimate_participant_count` requires a dataframe (df) and a statement number to be passed in. This function is called within another named `generate_statement_dataframe` and requires a dataframe of a specific format to be supplied in as well as a dataframe named `statements` to have been instantiated before hand. This function should not be called in isolation.

The purpose of these functions is to return the forecast participant count for the provided statement number.


In [ ]:
# Define function for producing a participant volume forecast for a specific statement number
def participant_volume_forecast(stm_number):
  # Get the average participant volume percentage for the statement number
  exp_percentages = (
      statement_participant_volume_percentage
      .loc[statement_participant_volume_percentage['statement_number']==stm_number]
  )
  # Get the maximum volume of participants from the 2023 cohort
  exp_participants = (
        max_participant_volumes[max_participant_volumes['cohort']==2023].copy()
    )

  # Use the average attrition values for ECTs & Mentors to predict the total participant count for the cohort using the maximum values seen in the 2023 cohort.
  exp_participants['est_participants'] = np.where(
      exp_participants['participant_type']=='ECT',
        np.ceil(exp_participants['max_participant_count']+
        (exp_participants['max_participant_count']*ect_avg_attrition)),

        np.ceil(exp_participants['max_participant_count']+
        (exp_participants['max_participant_count']*mentor_avg_attrition))).astype(int)

  # Combine the forecast total participants with the percentages per statement number to calculate the forecast per statement number
  exp_participants = exp_participants.merge(exp_percentages.rename(columns={'adj_participant_percentage':'forecast_participant_percentage'}), how='left', on='participant_type')

  exp_participants['forecast_participants'] = np.ceil(exp_participants['est_participants'] * (exp_participants['forecast_participant_percentage']/100)).astype(int)

  # Return the dataframe
  return exp_participants[['participant_type', 'forecast_participant_percentage', 'forecast_participants', 'est_participants']].rename(columns={'est_participants':'forecast_total_participants'})

In [ ]:
def estimate_participant_count(df, stm_number):
  exp_percentages = (
      statement_participant_volume_percentage
  )

  # Statement 1 Logic
  ## Statement 1 forecast has no previous data to build upon and therefore will calculate the forecast participants based on average cohort attrition and the average percentage present.
  if stm_number == 1:
    exp_participants = participant_volume_forecast(1)

  # Statement 2 Logic
  ## Statement 2 may have actual values from statement 1 or not. If there are values in statement 1 it rebases accordingly otherwise it uses the same logic as per statement 1 above.
  if stm_number == 2:
    if statements[statements['statement_number']==1]['statement_has_passed'].iloc[0] == True:
      # Get the actual number recorded in Statement 1
      s1_participants = get_actual_participant_counts(1)
      # Merge on the percentage that the actual number is likely to represent.
      s1_participants = (
          s1_participants
          .merge(exp_percentages[['statement_number', 'participant_type', 'adj_participant_percentage']],
                how='left',
                on=['statement_number', 'participant_type']
          )
      )
      # Calculate the updated total participant count for the cohort based on the actual volume seen in statement 1
      s1_participants['forecast_total_participants'] = (
          np.ceil(s1_participants['actual_participants'] / (s1_participants['adj_participant_percentage']/100)).astype(int)
      )

      # Calculate the volume for Statement 2 using the average percentage expected against the updated total forecast volume.
      exp_participants = s1_participants[['participant_type', 'forecast_total_participants']].merge(exp_percentages[exp_percentages['statement_number']==stm_number].rename(columns={'adj_participant_percentage':'forecast_participant_percentage'}), how='left', on='participant_type')

      exp_participants['forecast_participants'] = np.ceil(exp_participants['forecast_total_participants'] * (exp_participants['forecast_participant_percentage']/100)).astype(int)

    else:
      exp_participants = participant_volume_forecast(2)

  # Statement 3 Logic
  ## Statement 3 may have 1 or 2 statements completed before it and as such the forecast participant volume must adapt. If Statement 1 has NOT passed, the attrition method is used. If Statemente 1 HAS passed then it assesses if Statement 2 has also. If Statement 2 has passed it will use the actual value, if it has not it will use the forecast value as this will have changed since Statement 1 has passed.
  if stm_number == 3:
    if statements[statements['statement_number']==1]['statement_has_passed'].iloc[0] == True:
      if statements[statements['statement_number']==2]['statement_has_passed'].iloc[0] == True:
        field_to_use = 'actual_participants'
      else:
        field_to_use = 'forecast_participants'

      # Calculate the new total participants expected
      s2_participants  = (
          generate_statement_dataframe(2)
      )

      s2_participants['forecast_total_participants'] = (
          np.ceil(s2_participants[field_to_use] / (s2_participants['forecast_participant_percentage']/100)).astype(int)
      )

      # Calculate the volume for Statement 3 using the average percentage expected against the updated total forecast volume.
      exp_participants = s2_participants[['participant_type', 'forecast_total_participants']].merge(exp_percentages[exp_percentages['statement_number']==stm_number].rename(columns={'adj_participant_percentage': 'forecast_participant_percentage'}), how='left', on='participant_type')

      exp_participants['forecast_participants'] = np.ceil(exp_participants['forecast_total_participants'] * (exp_participants['forecast_participant_percentage']/100)).astype(int)

    else:
      exp_participants = participant_volume_forecast(3)

  # # Statement 4 Logic
  # if stm_number == 4:
  #   if statements[statements['statement_number']==1]['statement_has_passed'].iloc[0] == True:
  #     if statements[statements['statement_number']==3]['statement_has_passed'].iloc[0] == True:
  #       field_to_use = 'actual_participants'
  #     else:
  #       field_to_use = 'forecast_participants'

  #     # Calculate the new total participants expected
  #     s3_participants  = (
  #         generate_statement_dataframe(3)
  #     )

  #     s3_participants['forecast_total_participants'] = (
  #         np.ceil(s3_participants[field_to_use] / (s3_participants['forecast_participant_percentage']/100)).astype(int)
  #     )

  #     exp_participants = s3_participants[['participant_type', 'forecast_total_participants']].merge(exp_percentages[exp_percentages['statement_number']==stm_number].rename(columns={'adj_participant_percentage': 'forecast_participant_percentage'}), how='left', on='participant_type')

  #     exp_participants['forecast_participants'] = np.ceil(exp_participants['forecast_total_participants'] * (exp_participants['forecast_participant_percentage']/100)).astype(int)

  #   else:
  #     exp_participants = participant_volume_forecast(4)

  ## Statement 3 is typically where we see the highest volume of participants registered in a cohort with the value decreasing over each subsequent statement. As such, the logic for statements 4 onwards check whether statement 1 has passed to see if it needs to use inner-cohort values or not and then if the previous statement has passed to determine if its actual or forecast value should be used.

  # Statement 4+ Logic
  if stm_number >= 4:
    if statements[statements['statement_number']==1]['statement_has_passed'].iloc[0] == True:
      if statements[statements['statement_number']==stm_number-1]['statement_has_passed'].iloc[0] == True:
        field_to_use = 'actual_participants'
      else:
        field_to_use = 'forecast_participants'

      prev_s_participants  = (
          generate_statement_dataframe(stm_number-1)
      )

      prev_s_participants['forecast_total_participants'] = (
          np.floor(prev_s_participants[field_to_use] / (prev_s_participants['forecast_participant_percentage']/100)).astype(int)
      )

      exp_participants = prev_s_participants[['participant_type', 'forecast_total_participants']].merge(exp_percentages[exp_percentages['statement_number']==stm_number].rename(columns={'adj_participant_percentage': 'forecast_participant_percentage'}), how='left', on='participant_type')

      exp_participants['forecast_participants'] = np.ceil(exp_participants['forecast_total_participants'] * (exp_participants['forecast_participant_percentage']/100)).astype(int)

    else:
      exp_participants = participant_volume_forecast(stm_number)

  # Merge the created columns onto the dataframe supplied when calling the function and return this.
  df = df.merge(exp_participants[['participant_type', 'forecast_participant_percentage', 'forecast_participants', 'forecast_total_participants']], how='left', on='participant_type')

  return df

#### Define Function for estimting declaration counts per Statement Number

**Outputs:** 3 Functions
1. `adj_declaration_volume_forecast(df)`
2. `declaration_volume_forecast(df)`
3. `estimate_declaration_counts`(df, stm_number)`

**Description:**

Within this section 3 functions are defined. The main function which is used by the `generate_statement_dataframe` function later is `estimate_declaration_counts`. The other two are used within this as supporting functions and to avoid repeating code with minor changes.

The purpose of these functions is to return a forecast volume of declarations per type and statement number. None of these functions should be called in isolation.

In [ ]:
def adj_declaration_volume_forecast(df):
  # Get the statement number from the df being modified
  stm_number = df['statement_number'].iloc[0]
  # Get the statement cohort from the df being modified
  cohort = statements['statement_cohort'].iloc[0]

  # Filter statement declaration volume percentages to the statement cohort gathered above
  adj_declaration_volume_percentage = (
      statement_declaration_volume_percentage
      .merge(statement_mappings[['statement_date', 'statement_number', 'statement_has_passed']].loc[statement_mappings['statement_cohort'] == cohort],
             how='left',
             on='statement_number')
  )
  # Filter to only those statements that have NOT passed or the statement number for the df being modified
  adj_declaration_volume_percentage = (
    adj_declaration_volume_percentage
    .loc[
        (adj_declaration_volume_percentage['statement_has_passed'] == False) |
        (adj_declaration_volume_percentage['statement_number']==stm_number)
        ]
    .copy()
  )

  # Calculate an adjusted declaration percentage for the remaining statements.
  ## THIS NEEDS A BETTER/LONGER EXPLANATION
  adj_declaration_volume_percentage['adj_declaration_percentage'] = round(
      (adj_declaration_volume_percentage['adj_declaration_percentage'] /
      adj_declaration_volume_percentage.groupby(['declaration_type', 'participant_type'])['adj_declaration_percentage'].transform('sum')) *100, 2
  )

  # Get the actual declaration count from the previous statements.
  previous_statement_declarations = (
      act_declarations[
          (act_declarations['statement_number']< stm_number) &
          (act_declarations['statement_date'] <= datetime.datetime.today())
          ]
  )

  # Calculate the Started Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  started_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='started'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='started'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='started']
          .groupby(['participant_type'])
          .agg(act_started_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  started_decs = (
      started_decs.assign(**{
          'fc_total_started_declarations':
          np.floor((started_decs['forecast_total_participants'] * (started_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  started_decs = (
      started_decs.assign(**{
          'fc_started_declarations':
          np.floor((started_decs['fc_total_started_declarations']-started_decs['act_started_declarations_td'])*(started_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  started_decs['fc_started_declarations'] = np.where(
      started_decs['fc_started_declarations'] <0, 0, started_decs['fc_started_declarations']
  )

  # Calculate the Retained-1 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ret1_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-1'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='retained-1'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='retained-1']
          .groupby(['participant_type'])
          .agg(act_ret1_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ret1_decs = (
      ret1_decs.assign(**{
          'fc_total_ret1_declarations':
          np.floor((ret1_decs['forecast_total_participants'] * (ret1_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ret1_decs = (
      ret1_decs.assign(**{
          'fc_ret1_declarations':
          np.floor((ret1_decs['fc_total_ret1_declarations']-ret1_decs['act_ret1_declarations_td'])*(ret1_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ret1_decs['fc_ret1_declarations'] = np.where(
      ret1_decs['fc_ret1_declarations'] < 0, 0, ret1_decs['fc_ret1_declarations']
  )

  # Calculate the Retained-2 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ret2_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-2'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='retained-2'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='retained-2']
          .groupby(['participant_type'])
          .agg(act_ret2_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ret2_decs = (
      ret2_decs.assign(**{
          'fc_total_ret2_declarations':
          np.floor((ret2_decs['forecast_total_participants'] * (ret2_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ret2_decs = (
      ret2_decs.assign(**{
          'fc_ret2_declarations':
          np.floor((ret2_decs['fc_total_ret2_declarations']-ret2_decs['act_ret2_declarations_td'])*(ret2_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ret2_decs['fc_ret2_declarations'] = np.where(
      ret2_decs['fc_ret2_declarations'] < 0, 0, ret2_decs['fc_ret2_declarations']
  )

  # Calculate the Retained-3 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ret3_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-3'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='retained-3'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='retained-3']
          .groupby(['participant_type'])
          .agg(act_ret3_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ret3_decs = (
      ret3_decs.assign(**{
          'fc_total_ret3_declarations':
          np.floor((ret3_decs['forecast_total_participants'] * (ret3_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ret3_decs = (
      ret3_decs.assign(**{
          'fc_ret3_declarations':
          np.floor((ret3_decs['fc_total_ret3_declarations']-ret3_decs['act_ret3_declarations_td'])*(ret3_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ret3_decs['fc_ret3_declarations'] = np.where(
      ret3_decs['fc_ret3_declarations'] < 0, 0, ret3_decs['fc_ret3_declarations']
  )

  # Calculate the Retained-4 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ret4_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-4'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='retained-4'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='retained-4']
          .groupby(['participant_type'])
          .agg(act_ret4_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ret4_decs = (
      ret4_decs.assign(**{
          'fc_total_ret4_declarations':
          np.floor((ret4_decs['forecast_total_participants'] * (ret4_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ret4_decs = (
      ret4_decs.assign(**{
          'fc_ret4_declarations':
          np.floor((ret4_decs['fc_total_ret4_declarations']-ret4_decs['act_ret4_declarations_td'])*(ret4_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ret4_decs['fc_ret4_declarations'] = np.where(
      ret4_decs['fc_ret4_declarations'] < 0, 0, ret4_decs['fc_ret4_declarations']
  )

  # Calculate the Extended-1 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ext1_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-1'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='extended-1'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='extended-1']
          .groupby(['participant_type'])
          .agg(act_ext1_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ext1_decs = (
      ext1_decs.assign(**{
          'fc_total_ext1_declarations':
          np.floor((ext1_decs['forecast_total_participants'] * (ext1_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ext1_decs = (
      ext1_decs.assign(**{
          'fc_ext1_declarations':
          np.floor((ext1_decs['fc_total_ext1_declarations']-ext1_decs['act_ext1_declarations_td'])*(ext1_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ext1_decs['fc_ext1_declarations'] = np.where(
      ext1_decs['fc_ext1_declarations'] < 0, 0, ext1_decs['fc_ext1_declarations']
  )

  # Calculate the Extended-2 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ext2_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-2'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='extended-2'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='extended-2']
          .groupby(['participant_type'])
          .agg(act_ext2_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ext2_decs = (
      ext2_decs.assign(**{
          'fc_total_ext2_declarations':
          np.floor((ext2_decs['forecast_total_participants'] * (ext2_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ext2_decs = (
      ext2_decs.assign(**{
          'fc_ext2_declarations':
          np.floor((ext2_decs['fc_total_ext2_declarations']-ext2_decs['act_ext2_declarations_td'])*(ext2_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ext2_decs['fc_ext2_declarations'] = np.where(
      ext2_decs['fc_ext2_declarations'] < 0, 0, ext2_decs['fc_ext2_declarations']
  )

  # Calculate the Extended-3 Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  ext3_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-3'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='extended-3'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='extended-3']
          .groupby(['participant_type'])
          .agg(act_ext3_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  ext3_decs = (
      ext3_decs.assign(**{
          'fc_total_ext3_declarations':
          np.floor((ext3_decs['forecast_total_participants'] * (ext3_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  ext3_decs = (
      ext3_decs.assign(**{
          'fc_ext3_declarations':
          np.floor((ext3_decs['fc_total_ext3_declarations']-ext3_decs['act_ext3_declarations_td'])*(ext3_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  ext3_decs['fc_ext3_declarations'] = np.where(
      ext3_decs['fc_ext3_declarations'] < 0, 0, ext3_decs['fc_ext3_declarations']
  )

  # Calculate the Completed Declaration volume by rebasing the total expected and then applying the expected % for the statement number.
  completed_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants','act_started_declaration_count']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='completed'],
          how='left', on='participant_type'
      )
      .merge(
          adj_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[adj_declaration_volume_percentage['declaration_type']=='completed'],
          how='left', on=['statement_number', 'participant_type']
      )
      .merge(
          previous_statement_declarations
          .loc[previous_statement_declarations['declaration_type']=='completed']
          .groupby(['participant_type'])
          .agg(act_completed_declarations_td = ('act_declaration_count', 'sum'))
          .reset_index(),
          how='left',
          on=['participant_type']
      )
      .fillna(0)
  )
  completed_decs = (
      completed_decs.assign(**{
          'fc_total_completed_declarations':
          np.floor((completed_decs['forecast_total_participants'] * (completed_decs['avg_declaration_proportion']/100))).astype(int)
      })
  )
  completed_decs = (
      completed_decs.assign(**{
          'fc_completed_declarations':
          np.floor((completed_decs['fc_total_completed_declarations']-completed_decs['act_completed_declarations_td'])*(completed_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )
  completed_decs['fc_completed_declarations'] = np.where(
      completed_decs['fc_completed_declarations'] < 0, 0, completed_decs['fc_completed_declarations']
  )


  # Combine all dataframes for each declaration type together so it creates one column per type
  df = (
      df
      .merge(started_decs[['participant_type', 'fc_started_declarations']], how='left', on='participant_type')
      .merge(ret1_decs[['participant_type', 'fc_ret1_declarations']], how='left', on='participant_type')
      .merge(ret2_decs[['participant_type', 'fc_ret2_declarations']], how='left', on='participant_type')
      .merge(ret3_decs[['participant_type', 'fc_ret3_declarations']], how='left', on='participant_type')
      .merge(ret4_decs[['participant_type', 'fc_ret4_declarations']], how='left', on='participant_type')
      .merge(ext1_decs[['participant_type', 'fc_ext1_declarations']], how='left', on='participant_type')
      .merge(ext2_decs[['participant_type', 'fc_ext2_declarations']], how='left', on='participant_type')
      .merge(ext3_decs[['participant_type', 'fc_ext3_declarations']], how='left', on='participant_type')
      .merge(completed_decs[['participant_type', 'fc_completed_declarations']], how='left', on='participant_type')
  )

  return df

In [ ]:
# Define the function for calculating the declaration volume based on the forecast number of participants. This function is used when no declarations have already been recieved in previous statements.
def declaration_volume_forecast(df):
  # Calculate the Started Declaration volume by using the total expected and then applying the expected % for the statement number.
  started_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='started'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='started'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  started_decs = (
      started_decs.assign(**{
          'fc_started_declarations':
          np.ceil((started_decs['forecast_total_participants'] * (started_decs['avg_declaration_proportion']/100))*(started_decs['adj_declaration_percentage']/100)
          ).astype(int)
      })
  )

  ret1_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-1'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='retained-1'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ret1_decs = (
      started_decs.assign(**{
          'fc_ret1_declarations':
          np.ceil((ret1_decs['forecast_total_participants']*(ret1_decs['avg_declaration_proportion']/100))*(ret1_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ret2_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-2'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='retained-2'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ret2_decs = (
      started_decs.assign(**{
          'fc_ret2_declarations':
          np.ceil((ret2_decs['forecast_total_participants']*(ret2_decs['avg_declaration_proportion']/100))*(ret2_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ret3_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-3'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='retained-3'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ret3_decs = (
      started_decs.assign(**{
          'fc_ret3_declarations':
          np.ceil((ret3_decs['forecast_total_participants']*(ret3_decs['avg_declaration_proportion']/100))*(ret3_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ret4_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='retained-4'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='retained-4'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ret4_decs = (
      ret4_decs.assign(**{
          'fc_ret4_declarations':
          np.ceil((ret4_decs['forecast_total_participants']*(ret4_decs['avg_declaration_proportion']/100))*(ret4_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ext1_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-1'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='extended-1'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ext1_decs = (
      ext1_decs.assign(**{
          'fc_ext1_declarations':
          np.ceil((ext1_decs['forecast_total_participants']*(ext1_decs['avg_declaration_proportion']/100))*(ext1_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ext2_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-2'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='extended-2'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ext2_decs = (
      ext2_decs.assign(**{
          'fc_ext2_declarations':
          np.ceil((ext2_decs['forecast_total_participants']*(ext2_decs['avg_declaration_proportion']/100))*(ext2_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  ext3_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='extended-3'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='extended-3'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  ext3_decs = (
      ext3_decs.assign(**{
          'fc_ext3_declarations':
          np.ceil((ext3_decs['forecast_total_participants']*(ext3_decs['avg_declaration_proportion']/100))*(ext3_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  completed_decs = (
      df[['statement_number', 'participant_type', 'forecast_total_participants']]
      .merge(
          avg_declaration_proportions[['participant_type', 'avg_declaration_proportion']]
          .loc[avg_declaration_proportions['declaration_type']=='completed'],
          how='left', on='participant_type'
      )
      .merge(
          statement_declaration_volume_percentage[['statement_number', 'participant_type', 'adj_declaration_percentage']]
          .loc[statement_declaration_volume_percentage['declaration_type']=='completed'],
          how='left', on=['statement_number', 'participant_type']
      )
      .fillna(0)
  )
  completed_decs = (
      completed_decs.assign(**{
          'fc_completed_declarations':
          np.ceil((completed_decs['forecast_total_participants']*(completed_decs['avg_declaration_proportion']/100))*(completed_decs['adj_declaration_percentage']/100)).astype(int)
      })
  )

  df = (
      df
      .merge(started_decs[['participant_type', 'fc_started_declarations']], how='left', on='participant_type')
      .merge(ret1_decs[['participant_type', 'fc_ret1_declarations']], how='left', on='participant_type')
      .merge(ret2_decs[['participant_type', 'fc_ret2_declarations']], how='left', on='participant_type')
      .merge(ret3_decs[['participant_type', 'fc_ret3_declarations']], how='left', on='participant_type')
      .merge(ret4_decs[['participant_type', 'fc_ret4_declarations']], how='left', on='participant_type')
      .merge(ext1_decs[['participant_type', 'fc_ext1_declarations']], how='left', on='participant_type')
      .merge(ext2_decs[['participant_type', 'fc_ext2_declarations']], how='left', on='participant_type')
      .merge(ext3_decs[['participant_type', 'fc_ext3_declarations']], how='left', on='participant_type')
      .merge(completed_decs[['participant_type', 'fc_completed_declarations']], how='left', on='participant_type')
  )

  return df

In [ ]:
def estimate_declaration_counts(df, stm_number):
  # If the statement being assessed is Statement 1 there will be no previous inner-cohort data to use, therefore it applies the average proportioning based on the number of forecast participants.
  if stm_number == 1:
    declaration_volumes = declaration_volume_forecast(df)

  # If the statement being assessed is 2 or higher then we check if Statement 1 has passed. If it has NOT passed, the average apportioning method is used otherwise, the expected percentage per statement needs to be adjusted as well as the expected volume to account for how many have already been recieved and how many statements they are expected to span going forward.
  if stm_number >=2:
    if statements[statements['statement_number']==1]['statement_has_passed'].iloc[0] == True:
      declaration_volumes = adj_declaration_volume_forecast(df)

    else:
      declaration_volumes = declaration_volume_forecast(df)

  return declaration_volumes

#### Generate Statement DataFrames

**Output:** Function `generate_statement_dataframe(stm_number)`

**Description:**

This section defines the above function which requires a statement number to be passed in. The function then builds a dataframe for the statement number provided by calling 4 sub functions defined earlier. The functions called in sequence are:
1. `get_actual_participant_counts`
2. `estimate_participant_count`
3. `get_actual_declarations`
4. `estimate_declaration_counts`

The function then returns the completed dataframe which will be concatenated as part of the function `combine_statement_dataframes()`.

In [ ]:
def generate_statement_dataframe(stm_number):
  df = get_actual_participant_counts(stm_number)

  df = estimate_participant_count(df, stm_number)

  df = get_actual_declarations(df, stm_number)

  df = estimate_declaration_counts(df, stm_number)

  return df

#### Combine Statement Dataframes

**Output:** Function `combine_statement_dataframes()`

**Description:**

The function defined in this section calls the `generate_statement_dataframe` function defined above, 11 times (once for each statement from number 1 to 11) and concatenates these into a final dataframe that is returned.

In [ ]:
def combine_statement_dataframes():
  dfs = []
  for i in range(1,12):
    df = generate_statement_dataframe(i)
    dfs.append(df)

  combined_df = pd.concat(dfs)

  return combined_df

## **Outputs**

### **Generate Forecasts**

**Description:**

This section contains the generated forecasts for the sepcified cohorts.

Each cohort forecast is generated using the functions defined above and produce a dataframe output which is later combined and sent to a BigQuery table for use in reporting.

#### 2023 Cohort

**Output:** `fc_2023`

In [ ]:
# Specify the cohort
cohort = 2023

# Filter the required data and store in the required variables for above functions to run.
statements = get_filtered_data(cohort)[0]
act_participants = get_filtered_data(cohort)[1]
act_declarations = get_filtered_data(cohort)[2]

# Call produce forecast function and store in a usable variable
fc_2023 = combine_statement_dataframes()

#### 2024 Cohort

**Output:** `fc_2024`

In [ ]:
# Specify the cohort
cohort = 2024

# Filter the required data and store in the required variables for above functions to run.
statements = get_filtered_data(cohort)[0]
act_participants = get_filtered_data(cohort)[1]
act_declarations = get_filtered_data(cohort)[2]

# Call produce forecast function and store in a usable variable
fc_2024 = combine_statement_dataframes()

### **Send Data to BigQuery**

In [ ]:
combined_data = pd.concat([fc_2023, fc_2024])

project_id = 'ecf-bq'
dataset_id = 'python_notebook_TP_Dev'
table_id = 'ecf_declaration_forecasts'

# Initialize BigQuery client
client = bigquery.Client(project=project_id)

# Full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Define the job configuration.
job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.

    # Set the write disposition. BigQuery appends loaded rows to an existing table by default, but with WRITE_TRUNCATE write disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)


# Create or update the table
job = client.load_table_from_dataframe(combined_data, full_table_id, job_config=job_config)
# Wait for the job to complete
job.result()

# Fetch the table
table = client.get_table(full_table_id)

# Update the table description
table.description = "This mart contains forecast declaration volumes for the 2023 & 2024 cohorts."

# Update the table metadata in BigQuery
client.update_table(table, ["description"])



Table(TableReference(DatasetReference('ecf-bq', 'python_notebook_TP_Dev'), 'ecf_declaration_forecasts'))